In [16]:
import pandas as pd
from pathlib import Path


In [17]:
#retrieve data from csv files and create data frames
school_data = Path("../Resources/schools_complete.csv")
student_data = Path("../Resources/students_complete.csv")
school_df = pd.read_csv(school_data)
student_df = pd.read_csv(student_data)

In [20]:
#merge the two dataframes and retrieve a header of the combined frame
combined_df = pd.merge(school_df, student_df, how = "left", on = ["school_name", "school_name"])
combined_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [33]:
#determine the number of unique schools within the data and return the amount.
unique_schools_count = len(pd.unique(combined_df['school_name']))
print(f"the number of unique schools is" + " " + str(unique_schools_count))

the number of unique schools is 15


In [46]:
#determine the number of students in the dataframe and return the amount
unique_students_count = len(pd.unique(student_df['Student ID']))
print(f"the number of unique students is" + " " + str(unique_students_count))

the number of unique students is 39170


In [32]:
#determine total budget of all schools in data frame
total_budget = school_df['budget'].sum()
print(f"the total budget of all schools recorded is" + " " + str(total_budget))


the total budget of all schools recorded is 24649428


In [54]:
#calculate the average math scores of students in the data frame
math_average = round(student_df['math_score'].mean(), 2)
print(f"the average math score of students in the data frame is" + " " + str(math_average))

the average math score of students in the data frame is 78.99


In [55]:
#calculate the average reading scores of students in the data frame
reading_average = round(student_df['reading_score'].mean(), 2)
print(f"the average reading score of students in the data frame is" + " " + str(reading_average))

the average reading score of students in the data frame is 81.88


In [59]:
#calculate the percentage of students passing math
passing_countM = student_df[(student_df['math_score'] >=70)].count()['student_name']
passing_rateM = round((passing_countM/unique_students_count)*100 ,2)
print(f"the percentage of students passing math is" + " " + str(passing_rateM))

the percentage of students passing math is 74.98


In [58]:
#calculate the percentage of students passing reading
passing_countR =  student_df[(student_df['reading_score'] >=70)].count()['student_name']
passing_rateR = round((passing_countR/unique_students_count)*100, 2)
print(f"the percent of students passing reading is" + " " + str(passing_rateR))

the percent of students passing reading is 85.81


In [62]:
#calculate the percentage that passed both math and reading
passing_both_count = student_df[
    (student_df['math_score'] >= 70) & (student_df['reading_score'] >= 70)
].count()['student_name']
overall_passing_rate = round((passing_both_count/unique_students_count)*100, 2)
print("the percentage of students that passed both reading and math is" + " " + str(overall_passing_rate))

the percentage of students that passed both reading and math is 65.17


In [74]:
#Source 1
#create a snapshot for the district's data summation 
district_summary = [{"Total Schools": unique_schools_count, 
            "Total Students": unique_students_count, 
            "Total Budget": total_budget, 
            "Average Math Score":  math_average, 
            "Average Reading Score":  reading_average, 
            "Math Pass Rate": passing_rateM,
            "Reading Pass Rate": passing_rateR,
            "Overall Pass Rate": overall_passing_rate}]
snapshot = pd.DataFrame(district_summary)

#Format new data frame 
#Source 2
snapshot["Total Budget"] = snapshot["Total Budget"].map("${:,}".format)
snapshot["Total Students"] = snapshot["Total Students"].map("{:,}".format)
snapshot["Math Pass Rate"] = snapshot["Math Pass Rate"].map("{:}%".format)
snapshot["Reading Pass Rate"] = snapshot["Reading Pass Rate"].map("{:}%".format)
snapshot["Overall Pass Rate"] = snapshot["Overall Pass Rate"].map("{:}%".format)


#show new data frame "snapshot"
snapshot

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


In [201]:
#Source 3
school_summary = combined_df.groupby(['school_name'])

school_names = combined_df.school_name.sort_values().unique()

#source 4
school_types = school_df.sort_values(by="school_name").type

students_per = list(school_summary.student_name.count())

school_budget = list(school_summary.budget.mean())
#Source Number 5

budget_per_capita = [i/j for i,j in zip(school_budget,students_per)]

per_school_M = list(school_summary.math_score.mean())

per_school_R = list(school_summary.reading_score.mean())

school_summary = combined_df[combined_df['math_score'] >= 70].groupby(['school_name'])

passing_math = [(i/j)*100 for i,j in zip(school_summary.math_score.count(),students_per)]

school_summary = combined_df[combined_df['reading_score'] >= 70].groupby(['school_name'])

passing_reading = [(i/j)*100 for i,j in zip(school_summary.reading_score.count(),students_per)]

passing_both = [(i+j)/2 for i,j in zip(passing_math,passing_reading)]

#Create new data frame for summary of per school data
school_summary_df = pd.DataFrame({"School Name":school_names,
                                  "School Type":school_types,
                                  "Total Students":students_per,
                                  "Total School Budget":school_budget,
                                  "Per Student Budget":budget_per_capita,
                                  "Average Math Score":per_school_M,
                                  "Average Reading Score":per_school_R,
                                  "Math Pass Rate":passing_math,
                                  "Reading Pass Rate":passing_reading,
                                  "Overall Pass Rate":passing_both})

school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,.0f}".format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)
school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].map("{:,.2f}".format)
school_summary_df["Average Reading Score"] = school_summary_df["Average Reading Score"].map("{:,.2f}".format)
school_summary_df["Math Pass Rate"] = school_summary_df["Math Pass Rate"].map("%{:,.2f}".format)
school_summary_df["Reading Pass Rate"] = school_summary_df["Reading Pass Rate"].map("%{:,.2f}".format)
school_summary_df["Overall Pass Rate"] = school_summary_df["Overall Pass Rate"].map("%{:,.2f}".format)

#Source 6
school_summary_df = school_summary_df.reset_index(drop=True)
school_summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,%66.68,%81.93,%74.31
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,%94.13,%97.04,%95.59
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,%65.99,%80.74,%73.36
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,%68.31,%79.30,%73.80
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,%93.39,%97.14,%95.27
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,%66.75,%80.86,%73.81
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,%92.51,%96.25,%94.38
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,%65.68,%81.32,%73.50
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,%66.06,%81.22,%73.64
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,%94.59,%95.95,%95.27


In [203]:
#Sort schools by overall passing rate in descending order
highest_performing = school_summary_df.sort_values(by= "Overall Pass Rate", ascending = False).reset_index(drop= True)
highest_performing.head() 

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
0,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,%94.13,%97.04,%95.59
1,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,%93.27,%97.31,%95.29
2,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,%93.39,%97.14,%95.27
3,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,%94.59,%95.95,%95.27
4,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,%93.87,%96.54,%95.20


In [204]:
#sort schools by lowest overall passing rate
lowest_performing = school_summary_df.sort_values(by= "Overall Pass Rate", ascending = True).reset_index(drop= True)
lowest_performing.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
0,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,%66.37,%80.22,%73.29
1,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,%65.99,%80.74,%73.36
2,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,%65.68,%81.32,%73.50
3,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,%66.06,%81.22,%73.64
4,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,%68.31,%79.30,%73.80


In [ ]:
#sort the average math scores by grade
average_by_grade = 

In [ ]:
#sort by the average reading scores for each grade


,0,1
0,Charter,School Name School Type Average Ma...
1,District,School Name School Type Average ...
